In [91]:
import cv2
import numpy as np

In [92]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [93]:
import os
path1="images_"+str(1)+".jpg"
img = cv2.imread(path1)
img = cv2.resize(img, None, fx=3, fy=3)
height, width, channels = img.shape


In [94]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [95]:
class_ids = []
confidences = []
boxes = []
for out in outs: #outs 형식: 1: x 2:y 3: width 4: height
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)  

In [96]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:#인덱스에 물체 종류 저장
        x, y, w, h = boxes[i]#사이즈 저장
        label = str(classes[class_ids[i]])#이름 불러오기
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) # 시작 끝점
        cv2.putText(img, label, (x, y +h), font, 1, color, 1)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()